# Notebook to create netcdf file with integrated parameters (new approach)

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import os

from tqdm import tqdm


## Datasets Preparation

In [ ]:
def datasets_preparation():

    ds_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_grid_T.nc')
    ds_bio_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_'  + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_biol_T.nc')
    ds_prod_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_'  + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_prod_T.nc') 
    
    ds = xr.open_dataset (ds_name)
    ds_bio = xr.open_dataset (ds_bio_name)
    ds_prod = xr.open_dataset (ds_prod_name)

    diat_i = ((ds_bio.diatoms.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    flag_i = ((ds_bio.flagellates.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])
    
    diat_prod_i = ((ds_prod.PPDIAT.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / diat_i    
    flag_prod_i = ((ds_prod.PPPHY.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27]) / flag_i

    return (diat_i, flag_i, diat_prod_i, flag_prod_i)


## Datasets Preparation 2

In [ ]:
def datasets_preparation2(name, units):

    dataset = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'],
        attrs=dict(description='Integrated quantity of interest',
        long_name =name,
        units=units))
        
    return (dataset)


## File Creation

In [ ]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/files/apr_jun_2025.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})
    temp.close()


## Data Collection

In [ ]:
dict_month = {'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'}

path = os.listdir('/results2/SalishSea/nowcast-green.202111/')

folders = [x for x in path if ((x[2:5]=='may' or (x[2:5]=='apr' and x[0:2] > '15') or x[2:5]=='jun') and (x[5:7] == '25'))]
if '17apr24.aside' in folders:
    folders.pop(folders.index('17apr24.aside')) # Removing an unnecessary directory (17apr24.aside)

indx_dates=(np.argsort(pd.to_datetime(folders, format="%d%b%y")))
folders = [folders[i] for i in indx_dates]

# Open the mesh mask
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
mask = mesh.tmask.to_numpy()

coords = dict(time_counter=pd.to_datetime(folders, format="%d%b%y"), y=mesh.y, x=mesh.x) 

diat_i_all = datasets_preparation2('Diatom Concentration', '[mmol m-2]')
flag_i_all = datasets_preparation2('Flagellate Concentration', '[mmol m-2]')

diat_prod_i_all = datasets_preparation2('Diatom Production Rate', '[mmol N m-2 s-1]')
flag_prod_i_all = datasets_preparation2('Flagellate Production Rate', '[mmol N m-2 s-1]')


In [ ]:
len(folders)

## Filling the arrays

In [ ]:
for i in tqdm(enumerate(folders)):

    diat_i, flag_i, diat_prod_i, flag_prod_i = datasets_preparation()

    diat_i_all[i[0]] = diat_i
    flag_i_all[i[0]] = flag_i

    diat_prod_i_all[i[0]] = diat_prod_i
    flag_prod_i_all[i[0]] = flag_prod_i
    

## Creating the datasets

In [ ]:
file_creation(diat_i_all, 'Diatom')
file_creation(flag_i_all, 'Flagellate')

file_creation(diat_prod_i_all, 'Diatom_Production_Rate')
file_creation(flag_prod_i_all, 'Flagellate_Production_Rate')
